# PROJECT 2 : Part 1
## Ames Housing Data and Kaggle Challenge

### Background
Real Sky Estate Development has been developing residential areas around Ames and is looking to procure and develop new housing in the area. 

Previous housing developments were not returning favourable profits due to the recent pandemic and political tensions. It also caused a rise in the costs of living and building materials over the years.

Facing a forecasted recession in the upcoming year, Real Sky senior management team has reached out to our data science team to pinpoint factors that will direct towards revamping the company’s focus structure to improve the attractiveness and sales price of new housing developments.  

### Problem Statement
What core features should we, Real Sky, focus on to increase the sale price of homes for our next development project?

## Data Cleaning
This notebook will focus on cleaning the given dataset in prepatory for EDA and visualisations (Part 2). 

In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import statistics as stats
import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer

### Data
Ames Housing Dataset:
train.csv & test.csv

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
# Load data
traindf = pd.read_csv('datasets/train.csv')

In [4]:
# Display train data

traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [6]:
traindf.shape

(2051, 81)

In [7]:
# Check for null values
null_stats = pd.DataFrame(traindf.isnull().sum(), columns = ['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count']/traindf.shape[0]*100,2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count',ascending=False)

,missing_value_count,% of dataset
Pool QC,2042,99.56
Misc Feature,1986,96.83
Alley,1911,93.17
Fence,1651,80.50
Fireplace Qu,1000,48.76
Lot Frontage,330,16.09
Garage Yr Blt,114,5.56
Garage Cond,114,5.56
Garage Qual,114,5.56
Garage Finish,114,5.56


27 columns have shown to have missing data. 6 of them have only 1 missing value each, while Pool QC only have 9 values in the data frame. 

The top 4 columns with less than 20% of values present will be dropped. There are too little data present. If those null values were filled it would skew the accuracy of the data. 
The columns with less than 1% missing values will have those rows containing the missing value removed. As it is just 1 row out of the 2051 rows, it would not affect the data accuracy by much or any at all. 

In [8]:
traindf.drop(columns=['Pool QC','Misc Feature', 'Alley', 'Fence'], inplace=True)



In [9]:
traindf.dropna(subset =['Bsmt Half Bath', 'Bsmt Full Bath', 'Total Bsmt SF', 'Bsmt Unf SF', 'BsmtFin SF 2', 'Garage Cars', 'Garage Area', 'BsmtFin SF 1'],inplace=True)

In [10]:
null_stats = pd.DataFrame(traindf.isnull().sum(), columns = ['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count']/traindf.shape[0]*100,2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count',ascending=False)

,missing_value_count,% of dataset
Fireplace Qu,998,48.73
Lot Frontage,330,16.11
Garage Type,113,5.52
Garage Yr Blt,113,5.52
Garage Finish,113,5.52
Garage Qual,113,5.52
Garage Cond,113,5.52
Bsmt Exposure,56,2.73
BsmtFin Type 2,54,2.64
Bsmt Qual,53,2.59


#### Replace missing values
Missing values will be handled based on whether it is a categorical value or numerical value as the processes would be different. The data will be split accordingly. 

For features involving the garage, as they all have the same number of null values, and with reference from the data dictionary provided, it is safe to say that the null value is equivalent to no garage, 'None'. These values will be replace with 'None'.

In [11]:
traindf[['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Qual', 'Garage Cond']] = traindf[['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Qual', 'Garage Cond']].fillna('None')

In [12]:
# Separate numerical and categorical features

trainCat = traindf.loc[:, traindf.dtypes==object]
trainNum = traindf.loc[:, traindf.dtypes!=object]

In [13]:
trainCat.shape

(2048, 39)

In [14]:
trainNum.shape

(2048, 38)

### Categorical values

In [15]:
null_stats = pd.DataFrame(trainCat.isnull().sum(), columns = ['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count']/trainCat.shape[0]*100,2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count',ascending=False)

,missing_value_count,% of dataset
Fireplace Qu,998,48.73
Bsmt Exposure,56,2.73
BsmtFin Type 2,54,2.64
Bsmt Qual,53,2.59
Bsmt Cond,53,2.59
BsmtFin Type 1,53,2.59
Mas Vnr Type,22,1.07


In [16]:
# Investigate the types of value under object-type columns

In [17]:
trainCat['Fireplace Qu'].value_counts()

Gd    522
TA    407
Fa     59
Po     31
Ex     31
Name: Fireplace Qu, dtype: int64

In [18]:
trainCat['Bsmt Exposure'].value_counts()

No    1338
Av     288
Gd     203
Mn     163
Name: Bsmt Exposure, dtype: int64

In [19]:
trainCat['BsmtFin Type 2'].value_counts()

Unf    1748
Rec      80
LwQ      60
BLQ      48
ALQ      35
GLQ      23
Name: BsmtFin Type 2, dtype: int64

In [20]:
trainCat['Bsmt Cond'].value_counts()

TA    1833
Gd      89
Fa      65
Po       5
Ex       3
Name: Bsmt Cond, dtype: int64

In [21]:
trainCat['Bsmt Qual'].value_counts()

TA    887
Gd    863
Ex    184
Fa     60
Po      1
Name: Bsmt Qual, dtype: int64

In [22]:
trainCat['BsmtFin Type 1'].value_counts()

GLQ    615
Unf    603
ALQ    292
BLQ    200
Rec    183
LwQ    102
Name: BsmtFin Type 1, dtype: int64

In [23]:
trainCat['Mas Vnr Type'].value_counts()

None       1215
BrkFace     630
Stone       168
BrkCmn       13
Name: Mas Vnr Type, dtype: int64

#### Fill replace ordinal values 
Some of the features have values where it can be a numerical. By replacing their values with numerical values it will greatly increase the accuracy in the machine learning process. These features will be filled with a rating numerical scale. Features to use fill-and-replace are: Fireplace Qu, Garage Cond, Garage Qual, Bsmt Exposure, Bsmt Cond, Bsmt Qual. 

In [24]:
# Create dictionary of replacement values

replace = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1 , np.NaN :0}
    
replace2 = {'Gd':4, 'Av':3, 'Mn':2, 'No':1, np.NaN :0}

In [25]:
# Fill and replace with appropriate dictionary
# Fireplace Qu
trainCat['Fireplace Qu'] = trainCat['Fireplace Qu'].map(replace)
# Bsmt Exposure 
trainCat['Bsmt Exposure'] = trainCat['Bsmt Exposure'].map(replace2)
# Bsmt Cond 
trainCat['Bsmt Cond'] = trainCat['Bsmt Cond'].map(replace)
# Bsmt Qual
trainCat['Bsmt Qual'] = trainCat['Bsmt Qual'].map(replace)

#### Fill the remaining null values

In [26]:
# Replace null values with 'none' in remaining columns

trainCat[['BsmtFin Type 2', 'BsmtFin Type 1', 'Mas Vnr Type']] = trainCat[['BsmtFin Type 2', 'BsmtFin Type 1', 'Mas Vnr Type']].fillna('None')

In [27]:
# Check Null values
null_stats = pd.DataFrame(trainCat.isnull().sum(), columns = ['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count']/trainCat.shape[0]*100,2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count',ascending=False)

,missing_value_count,% of dataset


### Numerical Values

In [28]:
# Check Null values
null_stats = pd.DataFrame(trainNum.isnull().sum(), columns = ['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count']/trainNum.shape[0]*100,2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count',ascending=False)

,missing_value_count,% of dataset
Lot Frontage,330,16.11
Mas Vnr Area,22,1.07


Lot Frontage has 16.11% missing values. This is relatively high. If these values are removed it might skew the data and decrease the accuracy. These values, both Lot Frontage and Mas Vnr Area, will have its values replaced by its respective mean value.

In [29]:
# Fill missing values with mean
# Lot Frontage
trainNum['Lot Frontage'] = trainNum['Lot Frontage'].fillna(np.mean(trainNum['Lot Frontage']))
# Mas Vnr Area
trainNum['Mas Vnr Area'] = trainNum['Mas Vnr Area'].fillna(np.mean(trainNum['Mas Vnr Area']))


In [30]:
# Check Null values
null_stats = pd.DataFrame(trainNum.isnull().sum(), columns = ['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count']/trainNum.shape[0]*100,2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count',ascending=False)

,missing_value_count,% of dataset


Both categorical and numerical data are now ready for the next part of the process. This will continue in Part 2. 

Both the data frames will continue to be split through the EDA and visualisation process as they would be analyse differently. 

In [31]:
# Save df as csv
trainNum.to_csv('datasets/trainNum.csv', index=False)
trainCat.to_csv('datasets/trainCat.csv', index=False)